<a href="https://colab.research.google.com/github/AndreMartins21/Analise-SISU-UFMG/blob/main/DF_SISU_UFMG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - LENDO O PDF, MANIPULANDO AS STRINGS E CRIANDO O DATAFRAME

In [2]:
# Importando as Bibliotecas

!pip install PyPDF2

import PyPDF2 as PDF
import pandas as pd
import matplotlib.pyplot as plt
import regex as re

**<> Lendo o arquivo PDF, e salvando todas as páginas**

In [5]:
arquivo = '/content/drive/MyDrive/Colab Notebooks/DataSet Kaggle/SISU UFMG.pdf'

lerPDF = PDF.PdfFileReader(arquivo)

num_page = lerPDF.numPages

paginas = []

# Filtrando os dados dos arquivos, separando somente os cursos / turnos / modalidade / máximo / mínimo
for cont in range(0, num_page):
  page = lerPDF.getPage(cont)

  conteudo = page.extractText()

  # SEPARAR SOMENTE OS CURSOS E SUAS NOTAS
  inicio = conteudo.find("MínimoMáximo") + len("MínimoMáximo")

  # Se for a última página:
  if cont == 15:
    fim = conteudo.find("334")
  else: 
    fim = conteudo.find("21/04/2021")
  
  paginas.append(conteudo[inicio:fim])

In [6]:
paginas[5]

'EDUCAÇÃO FÍSICA (Bacharelado)NoturnoAmplaconcorrência682,38768,30EDUCAÇÃO FÍSICA (Licenciatura)Matutino1.1460,60460,60EDUCAÇÃO FÍSICA (Licenciatura)Matutino1.2602,58628,22EDUCAÇÃO FÍSICA (Licenciatura)Matutino2.1436,00436,00EDUCAÇÃO FÍSICA (Licenciatura)Matutino2.2640,08662,94EDUCAÇÃO FÍSICA (Licenciatura)Matutino3.1499,32499,32EDUCAÇÃO FÍSICA (Licenciatura)Matutino3.2633,60671,58EDUCAÇÃO FÍSICA (Licenciatura)Matutino4.1589,92589,92EDUCAÇÃO FÍSICA (Licenciatura)Matutino4.2649,88677,42EDUCAÇÃO FÍSICA (Licenciatura)MatutinoAmplaconcorrência661,08689,90ENFERMAGEMMatutino1.1608,68608,68ENFERMAGEMMatutino1.2686,64718,28ENFERMAGEMMatutino2.1616,88616,88ENFERMAGEMMatutino2.2707,40722,92ENFERMAGEMMatutino3.1595,68595,68ENFERMAGEMMatutino3.2684,62735,64ENFERMAGEMMatutino4.1657,20657,20ENFERMAGEMMatutino4.2723,04740,42ENFERMAGEMMatutinoAmplaconcorrência727,52761,14ENGENHARIA AEROESPACIALMatutino1.1596,10596,10ENGENHARIA AEROESPACIALMatutino1.2699,08756,56ENGENHARIA AEROESPACIALMatutino2.1548,78

In [7]:
# SEPARANDO OS VALORES DE CADA COLUNA DO DATAFRAME

def sep_curso(string):
  
  # "6CIÊNCIAS CONTÁBEISNoturno"
  # Curso com maior nome tem 49 letras
  
  curso = r'[A-ZÍÃÓÇÔÊÉÁÂÚ\s()(Bacharelado)(Licenciatura)]{0,49}[VNMI]'
  find_curso = re.findall(curso, string)
  find_curso = list([x[:-1] for x in find_curso])
  return find_curso


def sep_turno(string):

  turno = r'Matutino|Noturno|Integral|Vespertino'

  find_turno = re.findall(turno, string)

  return find_turno

def sep_modalidade(string):
  #1.1 | Amplaconcorrência

  modalidade = r'\d{1}\.\d{1}|Amplaconcorrência'

  find_modalidade = re.findall(modalidade, string)

  return find_modalidade

def sep_minimo(string):
  #520,04

  minimo = r'\d{3},\d{3}'

  find_min = re.findall(minimo, string)
  find_min = list([x[:6] for x in find_min])

  return find_min

def sep_maximo(string):
  #04520

  maximo = r'\d{5},\d{2}'

  find_max = re.findall(maximo, string)
  find_max = list([x[2:] for x in find_max])

  return find_max


In [8]:
def data_frame(line):
  '''
  Função que retornará cada página do pdf em formato de DataFrame
  '''
  
  line = paginas[line]

  df_sisu = pd.DataFrame({
      "Curso": sep_curso(line),
      "Turno": sep_turno(line),
      "Modalidade": sep_modalidade(line),
      "Mínimo": sep_minimo(line),
      "Máximo": sep_maximo(line)
  })

  return df_sisu

data_frame(4).head()

,Curso,Turno,Modalidade,Mínimo,Máximo
0,CONTROLADORIA E FINANÇAS,Matutino,3.1,"557,82","557,82"
1,CONTROLADORIA E FINANÇAS,Matutino,3.2,"647,38","689,68"
2,CONTROLADORIA E FINANÇAS,Matutino,4.2,"717,08","761,82"
3,CONTROLADORIA E FINANÇAS,Matutino,Amplaconcorrência,"716,56","781,84"
4,DESIGN,Noturno,1.1,"578,92","578,92"


# TRANSFORMANDO O ARQUIVO EM DATAFRAME E PASSANDO PARA .XLSX

In [12]:
# Montando o dataframe completo

df = data_frame(0)

for cont in range(1, 16):
  df = df.append(data_frame(cont), ignore_index= True)

# Transformando os valores de MÁXIMO e MÍNIMO em float

df["Máximo"] = df["Máximo"].apply(lambda x: float(x.replace(",", ".")))
df["Mínimo"] = df["Mínimo"].apply(lambda x: float(x.replace(",", ".")))

df.tail()

,Curso,Turno,Modalidade,Mínimo,Máximo
679,ZOOTECNIA,Matutino,2.2,622.36,636.70
680,ZOOTECNIA,Matutino,3.1,494.06,494.06
681,ZOOTECNIA,Matutino,3.2,595.62,650.50
682,ZOOTECNIA,Matutino,4.2,626.86,646.20
683,ZOOTECNIA,Matutino,Amplaconcorrência,643.94,689.36


In [13]:
# Transformando em arquivo Excel:

df.to_excel("DF_SISU_UFMG.xlsx")